[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebooks/02_TVM_Tutorial_Relay.ipynb)

Please follow the introduction of the TVM tutorial before running this. 
The below code assumes you have already setup TVM, and merely loads it from your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
%%bash
[[ ! -e /tools/google-cloud-sdk ]] && exit
echo "Installing Dependencies ..."
sudo apt-get update
sudo apt-get install -y -q llvm-6.0 libglfw3-dev libtinfo-dev libffi-dev zlib1g-dev clinfo
cd "/content/gdrive/My Drive"
if [[ ! -e tvm ]]; then
    echo "Cloning TVM ..."
    git clone --recursive https://github.com/dmlc/tvm
fi
echo "Configuring Build ..."
cd tvm
mkdir -p build
cp cmake/config.cmake build
# sed -i -e 's/USE_OPONGL OFF/USE_OPONGL ON/g' build/config.cmake
sed -i -e 's/USE_CUDA OFF/USE_CUDA ON/g' build/config.cmake
sed -i -e 's/USE_CUDNN OFF/USE_CUDNN ON/g' build/config.cmake
sed -i -e 's/USE_LLVM OFF/USE_LLVM ON/g' build/config.cmake
sed -i -e 's/USE_VTA_TSIM OFF/USE_VTA_TSIM ON/g' build/config.cmake
cat build/config.cmake
echo "Running CMake ..."
cd build
cmake ..
echo "Building TVM ..."
make -j4
cd ..
echo "Installing Python libraries ..."
cd "/content/gdrive/My Drive/tvm/"
cd python; python setup.py install; cd ..
cd topi/python; python setup.py install

# Relay: an extensible deep learning IR

TODO fill in text